In [1]:
import pattern
from pattern.web import URL, DOM, plaintext

from os import listdir
from os.path import isfile, join
import os

import pandas as pd
import numpy as np
import nltk
from collections import Counter, defaultdict
from nltk.corpus import stopwords
import string
import random

filedic = {}
for each in os.listdir('lyrics'):
    if each != '.DS_Store':
        filedic[each] = listdir('lyrics/'+each)[1:]

In [5]:
lyricsdic = defaultdict(list)
exclude = set(string.punctuation)
lyricslist = []

remove_list = ['verse','hook','chorus', '','choruslloyd banks', '50 cent', 'outro','bridge', 'chorus', 'lyrics', 'lloyd banks', 'rick ross','lil wayne', 'da', 'dem', 'im', 'ya', 'dat', 'tony yayo', 'banks yay 50',
              'verse 1 tony yayo', 'verse 4 lloyd banks', 'verse 5 50 cent', 'verse 2 tony yayo', 'verse 1 50 cent', 'chorus 50 cent', 'intro 50 cent', 'biggie sample  in the background throughout the song', 'verse 3  tony yayo', 'verse 1  50 cent','chorus  50 cent','verse 2  lloyd banks','verse 2 lloyd banks', 'verse 6 tony yayo', 'verse 4 yony yayo']

for k,v in filedic.iteritems():
    for link in v:
        if link != '.ipynb_checkpoints':
            page = open('lyrics/'+k+'/'+link).read()
            dom = DOM(page)
            lyrics = dom('.lyrics')[0]
            p = plaintext(lyrics.content)
            p = ''.join(ch for ch in p if ch not in exclude)
            lyricslist.append(p)
            p = p.splitlines()
            lyricsdic[k].append(filter(lambda x: x.lower() not in remove_list, p))

In [6]:
lineslist = []
remove_list = ['verse','hook','chorus', '','choruslloyd banks', '50 cent', 'outro','bridge', 'chorus', 'lyrics', 'lloyd banks', 'rick ross','lil wayne', 'da', 'dem', 'im', 'ya', 'dat', 'tony yayo', 'banks yay 50',
              'verse 1 tony yayo', 'verse 4 lloyd banks', 'verse 5 50 cent', 'verse 2 tony yayo', 'verse 1 50 cent', 'chorus 50 cent', 'intro 50 cent', 'biggie sample  in the background throughout the song', 'verse 3  tony yayo', 'verse 1  50 cent','chorus  50 cent','verse 2  lloyd banks','verse 2 lloyd banks', 'verse 6 tony yayo', 'verse 4 yony yayo']
exportlist = []
for each in lyricslist:
    lineslist.append(each.splitlines())
lineslist = np.array(lineslist)
for each in lineslist:
    exportlist.append(filter(lambda x: x.lower() not in remove_list, each))

In [7]:
# import pickle
# f = open('picklelyrics', 'wb')

# pickle.dump(lyricsdic, f)

# # import simplejson
# # f = open('lyricsarchive.txt', 'w')
# # simplejson.dump(exportlist, f)
# # f.close()

In [25]:
tokenlist = []

for lyrics in lyricslist[:100]:
    tokens = nltk.word_tokenize(lyrics)
    tokens = [w for w in tokens if w.lower() not in ['50','cent','chief','keef', 'lloyd', 'banks', 'busta','rhymes']]
    tokenlist.extend([w for w in tokens if w.lower() not in stopwords.words('english')])

import random

class Markov(object):

    def __init__(self, words):
        self.cache = {}
        #self.open_file = open_file
        self.words = words
        self.word_size = len(self.words)
        self.database()

    def file_to_words(self):
        self.open_file.seek(0)
        data = self.open_file.read()
        words = data.split()
        return words

    def triples(self):
        if len(self.words) < 3:
            return
        for i in range(len(self.words) - 2):
            yield (self.words[i], self.words[i+1], self.words[i+2])

    def database(self):
        for w1, w2, w3 in self.triples():
            key = (w1, w2)
            if key in self.cache:
                self.cache[key].append(w3)
            else:
                self.cache[key] = [w3]

    def generate_markov_text(self, size=25):
        seed = random.randint(0, self.word_size-3)
        seed_word, next_word = self.words[seed], self.words[seed+1]
        w1, w2 = seed_word, next_word
        gen_words = []
        for i in xrange(size):
            gen_words.append(w1)
            w1, w2 = w2, random.choice(self.cache[(w1, w2)])
        gen_words.append(w2)
        return ' '.join(gen_words)

In [26]:
mark = Markov(tokenlist)
mark.generate_markov_text(size = 50)

u'Dont start lying tire flat catching allergy attack Ill axe battle bat Saddling hos like yeeehaw Bitch enjoy tour youre fuckin Doc 5 Oclock free ride Ambulance late paper thin Hook Bun B Pass cigar Ill bust open Ill dump Dig bag tricks pull lil somethin hold medical marijuana thats break'

In [27]:

import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from collections import Counter
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import linkage, dendrogram

vectorizer = TfidfVectorizer(stop_words='english', max_df = .6)
X = vectorizer.fit_transform(lyricslist)
features = vectorizer.get_feature_names()
kmeans = KMeans()
kmeans.fit(X)

top_centroids = kmeans.cluster_centers_.argsort()[:,-1:-6:-1]
print "top features for each cluster:"
for num, centroid in enumerate(top_centroids):
    print "%d: %s" % (num, ", ".join(features[i] for i in centroid))

top features for each cluster:
0: busta, rhymes, yo, yall, flipmode
1: 50, cent, banks, lloyd, hook
2: based, basedgod, bitches, god, lil
3: hook, em, man, money, ya
4: wayne, ya, money, lil, baby
5: chief, keef, sosa, hook, bang
6: lil, man, feel, real, love
7: cube, ice, big, yo, ll


In [28]:
def rhyme(inp, level):
    entries = nltk.corpus.cmudict.entries()
    syllables = [(word, syl) for word, syl in entries if word == inp]
    rhymes = []
    for (word, syllable) in syllables:
        rhymes += [word for word, pron in entries if pron[-level:] == syllable[-level:]]
    return set(rhymes)


In [32]:
tokenlines = []
wordtypes = defaultdict(list)

for each in lyricsdic['mfdoom']:
    for line in each:
        tokens = nltk.word_tokenize(line)
        struct = nltk.pos_tag(tokens)
        for each in struct:
            wordtypes[each[1]].append(each[0])
        tokenlines.append(','.join([x[1] for x in struct]))

sf = []
for each in lyricsdic['mfdoom'][0]:
    l = nltk.word_tokenize(each)
    t = nltk.pos_tag(l)
    sf.extend([x[1] for x in t])
    sf.extend('\n ')
sf = ' '.join(sf)
for each in sf.splitlines():
    sentence = ""
    for a in each.split():
        sentence +=  random.choice(wordtypes[a]).lower() + " "
    print sentence

like ho in his back bust over the cop everything it 
well was the tricks i could duke him drawls keep 
intro their business escaped 
it aint she lit operation spot 
and my back my return i see last 
yodleehheeewho kneed my ju herb stopper whole its fake go 
doom jaw power spell care so they killed this man cage 
and i are you buggin guns the dude’ in effect 
the speakeasy hes not sister 
magnum the much back 
on he more that back trunk need leash 
open hear song your tapes verse 
always i wanted square effects him found later 
and gun tape 
package of the tin in disease fuel feet 
im me like a nic flow 
im this worst notch 
globe told cats in monkey no home so wide pockets 
stashes of the nation of in out whole cup rappers 
or i slithered follow crucifix it robbed the rhymes 
nor cremate gym what her know yet in jakes on wrong rhymes 
watson it ghetto this razorthin to get to talk 
verse a bind lines to get a pen summer 
his then laugh sun of they 
your every girlfriend teeth 
just uh 

In [33]:
Counter(tokenlines).most_common(25)
sentence = ""

for each in ['PRP','VBD','DT','NN','PRP','VBD','DT','NN']:
    sentence +=  random.choice(wordtypes[each]).lower() + " "

print sentence
# last_word = sentence.rsplit(None, 1)[-1]
print ""

for each in ['PRP$','NNS','CC','PRP$','NN','NNP']:
     print random.choice(wordtypes[each]).lower(),

i got a glass they capped a breakfast 

my fools and his lame yours


In [34]:
postag = []
for line in tokenlines[0:10]:
    tags = nltk.pos_tag(line)
    #print tags
    #postag.append([[x[1] for x in tags]])